In [60]:
import pandas as pd
import itertools
import time
import tracemalloc

def compute_total_score(pairing, compatibility_matrix):
    """Computes the total compatibility score for a given pairing."""
    return sum(compatibility_matrix.get(pair, 0) for pair in pairing)

def branch_and_bound(compatibility_matrix, students):
    """Branch and Bound algorithm to find the optimal roommate pairings."""
    best_pairing = None
    best_score = float('-inf')
    num_allocations = 0

    # Lower bound: Calculate a naive solution (e.g., random pairings or sequential pairings)
    def compute_lower_bound():
        """Compute a quick lower bound using sequential pairings."""
        sorted_students = sorted(students)  # Sort students to ensure determinism
        pairs = [(sorted_students[i], sorted_students[i + 1]) for i in range(0, len(sorted_students) - 1, 2)]
        return compute_total_score(pairs, compatibility_matrix)

    lower_bound = compute_lower_bound()

    def search(remaining, current_pairs):
        nonlocal best_score, best_pairing, num_allocations

        # Base case: If all students are paired, compute the total score
        if not remaining:
            total_score = compute_total_score(current_pairs, compatibility_matrix)
            if total_score > best_score:
                best_score = total_score
                best_pairing = current_pairs
            return

        # Select first student
        first = remaining[0]

        # Try pairing them with each possible partner
        for partner in remaining[1:]:
            new_pair = (first, partner)
            new_remaining = [p for p in remaining if p not in new_pair]

            # Compute an upper bound (greedy best possible pairings left)
            greedy_score = sum(sorted([compatibility_matrix.get(p, 0) for p in itertools.combinations(new_remaining, 2)], reverse=True)[:len(new_remaining)//2])
            current_score = compute_total_score(current_pairs, compatibility_matrix) + compatibility_matrix.get(new_pair, 0)

            # Prune if the max possible score from this branch is lower than the best-known solution or the lower bound
            if current_score + greedy_score < max(best_score, lower_bound):
                continue

            # Recurse with the new pairing
            num_allocations += 1
            search(new_remaining, current_pairs + [new_pair])

    search(students, [])
    return best_pairing, best_score, num_allocations

def exhaustive_search(compatibility_matrix, students):
    """Exhaustive search algorithm to find the optimal roommate pairings by checking all combinations."""
    best_pairing = None
    best_score = float('-inf')
    num_allocations = 0

    # Generate all possible pairings
    all_pairings = list(itertools.permutations(students))
    
    for perm in all_pairings:
        pairs = [(perm[i], perm[i+1]) for i in range(0, len(perm) - 1, 2)]
        total_score = compute_total_score(pairs, compatibility_matrix)
        num_allocations += 1

        if total_score > best_score:
            best_score = total_score
            best_pairing = pairs
    
    return best_pairing, best_score, num_allocations

def solve_roommate_matching(csv_path, method='branch_and_bound'):
    """Reads in compatibility data and finds the optimal pairing using the selected method."""
    # Load data
    df = pd.read_csv(csv_path)
    students = list(set(df['Student 1'].tolist() + df['Student 2'].tolist()))
    compatibility_matrix = {(row['Student 1'], row['Student 2']): row['Compatibility Score'] for _, row in df.iterrows()}

    # Ensure symmetry
    for (s1, s2), score in list(compatibility_matrix.items()):
        compatibility_matrix[(s2, s1)] = score

    # Start tracking memory and time
    tracemalloc.start()
    start_time = time.time()

    # Solve using the selected method
    if method == 'branch_and_bound':
        best_pairing, best_score, num_allocations = branch_and_bound(compatibility_matrix, students)
    elif method == 'exhaustive':
        best_pairing, best_score, num_allocations = exhaustive_search(compatibility_matrix, students)
    else:
        raise ValueError("Invalid method. Choose 'branch_and_bound' or 'exhaustive'")

    # Stop tracking
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Print results
    print(f"Optimal Pairing: {best_pairing}")
    print(f"Total Compatibility Score: {best_score}")
    print(f"Number of Allocations: {num_allocations}")
    print(f"Time Taken: {end_time - start_time:.4f} seconds")
    print(f"Peak Memory Usage: {peak / 10**6:.2f} MiB")

    return best_pairing, best_score, num_allocations, end_time - start_time, peak / 10**6

In [37]:
# Example usage:
solve_roommate_matching("data/compatibility_10.csv", method='branch_and_bound')

Optimal Pairing: [('Student_9', 'Student_6'), ('Student_3', 'Student_8'), ('Student_4', 'Student_1'), ('Student_7', 'Student_10'), ('Student_2', 'Student_5')]
Total Compatibility Score: 3.734657113884437
Number of Allocations: 68
Time Taken: 0.0046 seconds
Peak Memory Usage: 0.00 MiB


([('Student_9', 'Student_6'),
  ('Student_3', 'Student_8'),
  ('Student_4', 'Student_1'),
  ('Student_7', 'Student_10'),
  ('Student_2', 'Student_5')],
 3.734657113884437,
 68,
 0.0046198368072509766,
 0.003185)

In [41]:
# Example usage:
solve_roommate_matching("data/compatibility_6.csv", method='exhaustive')

Optimal Pairing: [('Student_5', 'Student_3'), ('Student_2', 'Student_1'), ('Student_4', 'Student_6')]
Total Compatibility Score: 2.2812001621554097
Number of Allocations: 720
Time Taken: 0.0040 seconds
Peak Memory Usage: 0.01 MiB


([('Student_5', 'Student_3'),
  ('Student_2', 'Student_1'),
  ('Student_4', 'Student_6')],
 2.2812001621554097,
 720,
 0.003964900970458984,
 0.006864)

In [43]:
solve_roommate_matching("data/compatibility_50.csv", method='branch_and_bound')

KeyboardInterrupt: 

In [21]:
# Example usage:
solve_roommate_matching("data/compatibility_20.csv", method='branch_and_bound')

Optimal Pairing: [('Student_4', 'Student_1'), ('Student_19', 'Student_10'), ('Student_5', 'Student_2'), ('Student_6', 'Student_14'), ('Student_8', 'Student_11'), ('Student_15', 'Student_18'), ('Student_12', 'Student_3'), ('Student_17', 'Student_20'), ('Student_7', 'Student_13'), ('Student_9', 'Student_16')]
Total Compatibility Score: 8.457752150968194
Number of Allocations: 2040
Time Taken: 0.2132 seconds
Peak Memory Usage: 0.01 MiB


([('Student_4', 'Student_1'),
  ('Student_19', 'Student_10'),
  ('Student_5', 'Student_2'),
  ('Student_6', 'Student_14'),
  ('Student_8', 'Student_11'),
  ('Student_15', 'Student_18'),
  ('Student_12', 'Student_3'),
  ('Student_17', 'Student_20'),
  ('Student_7', 'Student_13'),
  ('Student_9', 'Student_16')],
 8.457752150968194,
 2040,
 0.21318697929382324,
 0.005451)

## More efficient

In [46]:
import pandas as pd
import itertools
import time
import tracemalloc

def compute_total_score(pairing, compatibility_matrix):
    """Computes the total compatibility score for a given pairing."""
    return sum(compatibility_matrix.get(pair, 0) for pair in pairing)

def branch_and_bound_improved(compatibility_matrix, students):
    """
    Improved branch and bound:
      - Carries along current_score to avoid recomputing sums.
      - Uses a quick upper bound: for each remaining student, add its maximum possible pairing score (dividing by 2).
      - Chooses the next student to pair based on a heuristic (most constrained student).
    """
    best_pairing = None
    best_score = float('-inf')
    num_allocations = 0

    # --- Lower bound: a simple sequential pairing ---
    def compute_lower_bound():
        sorted_students = sorted(students)
        pairs = [(sorted_students[i], sorted_students[i + 1]) for i in range(0, len(sorted_students) - 1, 2)]
        score = sum(compatibility_matrix.get(pair, 0) for pair in pairs)
        return score, pairs

    lower_bound_score, lower_bound_pairs = compute_lower_bound()
    best_score = lower_bound_score
    best_pairing = lower_bound_pairs

    # --- Upper bound function ---
    def upper_bound(remaining, current_score):
        """
        For each student in 'remaining', add the maximum possible pairing score with any other student,
        then divide by 2 (since every pair is counted twice). Add that to current_score.
        """
        if len(remaining) < 2:
            return current_score
        additional = 0
        for i in range(len(remaining)):
            best_val = 0
            for j in range(len(remaining)):
                if i != j:
                    score = compatibility_matrix.get((remaining[i], remaining[j]), 0)
                    if score > best_val:
                        best_val = score
            additional += best_val
        return current_score + additional / 2

    # --- Recursive search ---
    def search(remaining, current_pairs, current_score):
        nonlocal best_score, best_pairing, num_allocations

        # Base case: All students paired
        if not remaining:
            if current_score > best_score:
                best_score = current_score
                best_pairing = current_pairs
            return

        # --- Heuristic: Choose the "most constrained" student ---
        # For each student, compute its best possible pairing value (with any other remaining student)
        best_candidate = None
        best_candidate_bound = float('inf')
        for student in remaining:
            candidate_max = 0
            for other in remaining:
                if student != other:
                    candidate_max = max(candidate_max, compatibility_matrix.get((student, other), 0))
            if candidate_max < best_candidate_bound:
                best_candidate_bound = candidate_max
                best_candidate = student

        first = best_candidate
        # Remove the chosen student from the remaining list
        remaining_after_first = remaining.copy()
        remaining_after_first.remove(first)

        # Try pairing 'first' with every possible partner
        for partner in remaining_after_first:
            new_pair = (first, partner)
            pair_score = compatibility_matrix.get(new_pair, 0)
            updated_score = current_score + pair_score
            updated_pairs = current_pairs + [new_pair]

            # Build new remaining list without the paired students
            updated_remaining = remaining_after_first.copy()
            updated_remaining.remove(partner)

            # Compute an upper bound for this branch
            ub = upper_bound(updated_remaining, updated_score)
            if ub < best_score:
                # Prune this branch because even in the best-case scenario it won't beat the current best.
                continue

            num_allocations += 1  # Count this pairing decision
            search(updated_remaining, updated_pairs, updated_score)

    # Start the recursive search
    search(students, [], 0)
    return best_pairing, best_score, num_allocations


def exhaustive_search(compatibility_matrix, students):
    """Exhaustive search algorithm to find the optimal roommate pairings by checking all combinations."""
    best_pairing = None
    best_score = float('-inf')
    num_allocations = 0

    # Generate all possible pairings
    all_pairings = list(itertools.permutations(students))
    
    for perm in all_pairings:
        pairs = [(perm[i], perm[i+1]) for i in range(0, len(perm) - 1, 2)]
        total_score = compute_total_score(pairs, compatibility_matrix)
        num_allocations += 1

        if total_score > best_score:
            best_score = total_score
            best_pairing = pairs
    
    return best_pairing, best_score, num_allocations

def solve_roommate_matching_improved(csv_path, method='branch_and_bound'):
    """Reads compatibility data from CSV and finds the optimal pairing using the improved branch-and-bound."""
    import pandas as pd
    import time
    import tracemalloc
    import itertools

    # Load compatibility data
    df = pd.read_csv(csv_path)
    students = list(set(df['Student 1'].tolist() + df['Student 2'].tolist()))
    compatibility_matrix = {
        (row['Student 1'], row['Student 2']): row['Compatibility Score'] for _, row in df.iterrows()
    }
    # Ensure symmetry
    for (s1, s2), score in list(compatibility_matrix.items()):
        compatibility_matrix[(s2, s1)] = score

    tracemalloc.start()
    start_time = time.time()
    
        # Solve using the selected method
    if method == 'branch_and_bound':
        best_pairing, best_score, num_allocations = branch_and_bound_improved(compatibility_matrix, students)
    elif method == 'exhaustive':
        best_pairing, best_score, num_allocations = exhaustive_search(compatibility_matrix, students)
    else:
        raise ValueError("Invalid method. Choose 'branch_and_bound' or 'exhaustive'")

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print("=== Improved Branch and Bound ===")
    print(f"Optimal Pairing: {best_pairing}")
    print(f"Total Compatibility Score: {best_score}")
    print(f"Number of Allocations (nodes expanded): {num_allocations}")
    print(f"Time Taken: {end_time - start_time:.4f} seconds")
    print(f"Peak Memory Usage: {peak / 10**6:.2f} MiB")

    return best_pairing, best_score, num_allocations, end_time - start_time, peak / 10**6

In [55]:
# Example usage:
solve_roommate_matching_improved("data/compatibility_10.csv", method='branch_and_bound')

=== Improved Branch and Bound ===
Optimal Pairing: [('Student_1', 'Student_8'), ('Student_10', 'Student_2'), ('Student_3', 'Student_4'), ('Student_6', 'Student_5'), ('Student_9', 'Student_7')]
Total Compatibility Score: 3.708931604508898
Number of Allocations (nodes expanded): 29
Time Taken: 0.0024 seconds
Peak Memory Usage: 0.00 MiB


([('Student_1', 'Student_8'),
  ('Student_10', 'Student_2'),
  ('Student_3', 'Student_4'),
  ('Student_6', 'Student_5'),
  ('Student_9', 'Student_7')],
 3.708931604508898,
 29,
 0.0024309158325195312,
 0.001984)

In [59]:
# Example usage:
solve_roommate_matching("data/compatibility_10.csv", method='exhaustive')

Optimal Pairing: [('Student_9', 'Student_7'), ('Student_3', 'Student_4'), ('Student_10', 'Student_2'), ('Student_8', 'Student_1'), ('Student_6', 'Student_5')]
Total Compatibility Score: 3.708931604508898
Number of Allocations: 3628800
Time Taken: 13.9426 seconds
Peak Memory Usage: 466.17 MiB


([('Student_9', 'Student_7'),
  ('Student_3', 'Student_4'),
  ('Student_10', 'Student_2'),
  ('Student_8', 'Student_1'),
  ('Student_6', 'Student_5')],
 3.708931604508898,
 3628800,
 13.942628860473633,
 466.170192)

In [61]:
# Example usage:
solve_roommate_matching("data/compatibility_10.csv", method='branch_and_bound')

Optimal Pairing: [('Student_9', 'Student_7'), ('Student_3', 'Student_4'), ('Student_10', 'Student_2'), ('Student_8', 'Student_1'), ('Student_6', 'Student_5')]
Total Compatibility Score: 3.708931604508898
Number of Allocations: 86
Time Taken: 0.0044 seconds
Peak Memory Usage: 0.00 MiB


([('Student_9', 'Student_7'),
  ('Student_3', 'Student_4'),
  ('Student_10', 'Student_2'),
  ('Student_8', 'Student_1'),
  ('Student_6', 'Student_5')],
 3.708931604508898,
 86,
 0.004411935806274414,
 0.002088)

In [80]:
# Example usage:
solve_roommate_matching_improved("data/compatibility_40.csv", method='branch_and_bound')

=== Improved Branch and Bound ===
Optimal Pairing: [('Student_2', 'Student_5'), ('Student_25', 'Student_6'), ('Student_11', 'Student_31'), ('Student_10', 'Student_8'), ('Student_12', 'Student_3'), ('Student_28', 'Student_4'), ('Student_15', 'Student_30'), ('Student_14', 'Student_22'), ('Student_9', 'Student_32'), ('Student_38', 'Student_19'), ('Student_13', 'Student_34'), ('Student_40', 'Student_7'), ('Student_26', 'Student_35'), ('Student_36', 'Student_33'), ('Student_17', 'Student_39'), ('Student_24', 'Student_20'), ('Student_23', 'Student_29'), ('Student_21', 'Student_18'), ('Student_27', 'Student_1'), ('Student_16', 'Student_37')]
Total Compatibility Score: 17.597011035262096
Number of Allocations (nodes expanded): 718872
Time Taken: 1476.8454 seconds
Peak Memory Usage: 0.02 MiB


([('Student_2', 'Student_5'),
  ('Student_25', 'Student_6'),
  ('Student_11', 'Student_31'),
  ('Student_10', 'Student_8'),
  ('Student_12', 'Student_3'),
  ('Student_28', 'Student_4'),
  ('Student_15', 'Student_30'),
  ('Student_14', 'Student_22'),
  ('Student_9', 'Student_32'),
  ('Student_38', 'Student_19'),
  ('Student_13', 'Student_34'),
  ('Student_40', 'Student_7'),
  ('Student_26', 'Student_35'),
  ('Student_36', 'Student_33'),
  ('Student_17', 'Student_39'),
  ('Student_24', 'Student_20'),
  ('Student_23', 'Student_29'),
  ('Student_21', 'Student_18'),
  ('Student_27', 'Student_1'),
  ('Student_16', 'Student_37')],
 17.597011035262096,
 718872,
 1476.845386981964,
 0.015456)

## New Try of more efficiency

In [81]:
import pandas as pd
import time
import tracemalloc

def solve_roommate_matching_improved(csv_path, method='branch_and_bound'):
    """
    Reads compatibility data from CSV and finds the optimal pairing using a
    branch and bound algorithm with an efficient bitmask-based representation.
    
    This version uses memoization for upper bound computations, avoids
    repeated list copying by representing the unmatched students as a bitmask,
    and sorts potential partners in descending order to improve pruning.
    
    Performance metrics (number of allocations and peak memory usage) are also reported.
    
    Parameters:
      csv_path: Path to CSV file containing columns "Student 1", "Student 2", and "Compatibility Score".
      method: Currently only 'branch_and_bound' is implemented.
    
    Returns:
      best_pairing_named: List of tuples with student names for the optimal pairing.
      best_score: Total compatibility score for the pairing.
      allocations: Number of recursive branch allocations (nodes expanded).
      run_time: Total time taken in seconds.
      peak_memory: Peak memory usage in MiB.
    """
    # Load compatibility data
    df = pd.read_csv(csv_path)
    
    # Build a sorted list of unique student names for consistent ordering.
    student_names = sorted(list(set(df['Student 1'].tolist() + df['Student 2'].tolist())))
    n = len(student_names)
    student_to_index = {name: i for i, name in enumerate(student_names)}
    index_to_student = {i: name for i, name in enumerate(student_names)}
    
    # Build an n x n compatibility matrix (list of lists), initializing all values to 0.0.
    compatibility = [[0.0 for _ in range(n)] for _ in range(n)]
    for _, row in df.iterrows():
        s1 = row['Student 1']
        s2 = row['Student 2']
        score = row['Compatibility Score']
        i = student_to_index[s1]
        j = student_to_index[s2]
        compatibility[i][j] = score
        compatibility[j][i] = score  # Ensure symmetry
    
    # Global variables for branch and bound.
    allocations = 0  # Count of nodes expanded.
    best_score = float('-inf')
    best_pairing = None
    memo = {}  # Memoization dictionary for upper-bound values for given bitmask states.
    
    def compute_upper_bound(mask):
        """
        Computes an upper bound on the additional score achievable from the set of unmatched students.
        For each student in the current mask, we add its best possible pairing value and then divide by 2
        (since each pairing is counted twice).
        """
        indices = [i for i in range(n) if mask & (1 << i)]
        additional = 0
        for i in indices:
            best_val = 0
            for j in indices:
                if i != j:
                    best_val = max(best_val, compatibility[i][j])
            additional += best_val
        return additional / 2

    def search(mask, current_score, current_pairs):
        nonlocal best_score, best_pairing, allocations
        
        # Base case: All students have been paired.
        if mask == 0:
            if current_score > best_score:
                best_score = current_score
                best_pairing = current_pairs.copy()
            return
        
        # Use memoization: If the best achievable additional score for this mask (from memo)
        # plus the current score does not beat the best_score, prune this branch.
        if mask in memo:
            if current_score + memo[mask] <= best_score:
                return
        else:
            ub_val = compute_upper_bound(mask)
            memo[mask] = ub_val
            if current_score + ub_val <= best_score:
                return
        
        # Choose the first student in the mask (lowest index).
        i = None
        for bit in range(n):
            if mask & (1 << bit):
                i = bit
                break
        
        # Remove student i from the mask.
        new_mask = mask & ~(1 << i)
        # Get a list of potential partners (indices) in the new mask.
        remaining_indices = [j for j in range(n) if new_mask & (1 << j)]
        # Sort potential partners in descending order of compatibility with i.
        remaining_indices.sort(key=lambda j: compatibility[i][j], reverse=True)
        
        # Try pairing student i with each potential partner j.
        for j in remaining_indices:
            pair_score = compatibility[i][j]
            updated_score = current_score + pair_score
            pair = (i, j)
            next_mask = new_mask & ~(1 << j)
            allocations += 1  # Count this pairing decision as an allocation.
            search(next_mask, updated_score, current_pairs + [pair])
    
    # Start timing and memory tracking.
    tracemalloc.start()
    start_time = time.time()
    
    full_mask = (1 << n) - 1  # Bitmask with all students unmatched.
    search(full_mask, 0, [])
    
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # Convert best_pairing (pairs of indices) to pairs of student names.
    if best_pairing is not None:
        best_pairing_named = [(index_to_student[i], index_to_student[j]) for (i, j) in best_pairing]
    else:
        best_pairing_named = None
    
    run_time = end_time - start_time
    peak_memory = peak / 10**6  # Convert bytes to MiB.
    
    print("=== Improved Branch and Bound (Bitmask) ===")
    print(f"Optimal Pairing: {best_pairing_named}")
    print(f"Total Compatibility Score: {best_score}")
    print(f"Number of Allocations (nodes expanded): {allocations/1000:.2f} thousand")
    print(f"Time Taken: {run_time:.4f} seconds")
    print(f"Peak Memory Usage: {peak_memory:.3f} MiB")
    
    return best_pairing_named, best_score, allocations, run_time, peak_memory

In [100]:
solve_roommate_matching_improved("data/compatibility_10.csv")

=== Improved Branch and Bound (Bitmask) ===
Optimal Pairing: [('Student_1', 'Student_8'), ('Student_10', 'Student_2'), ('Student_3', 'Student_4'), ('Student_5', 'Student_6'), ('Student_7', 'Student_9')]
Total Compatibility Score: 3.708931604508898
Number of Allocations (nodes expanded): 0.08 thousand
Time Taken: 0.0021 seconds
Peak Memory Usage: 0.006 MiB


([('Student_1', 'Student_8'),
  ('Student_10', 'Student_2'),
  ('Student_3', 'Student_4'),
  ('Student_5', 'Student_6'),
  ('Student_7', 'Student_9')],
 3.708931604508898,
 79,
 0.002087831497192383,
 0.006296)

In [99]:
solve_roommate_matching_improved("data/compatibility_40.csv")

=== Improved Branch and Bound (Bitmask) ===
Optimal Pairing: [('Student_1', 'Student_27'), ('Student_10', 'Student_8'), ('Student_11', 'Student_31'), ('Student_12', 'Student_3'), ('Student_13', 'Student_34'), ('Student_14', 'Student_22'), ('Student_15', 'Student_30'), ('Student_16', 'Student_37'), ('Student_17', 'Student_39'), ('Student_18', 'Student_21'), ('Student_19', 'Student_38'), ('Student_2', 'Student_5'), ('Student_20', 'Student_24'), ('Student_23', 'Student_29'), ('Student_25', 'Student_6'), ('Student_26', 'Student_35'), ('Student_28', 'Student_4'), ('Student_32', 'Student_9'), ('Student_33', 'Student_36'), ('Student_40', 'Student_7')]
Total Compatibility Score: 17.597011035262096
Number of Allocations (nodes expanded): 24272.19 thousand
Time Taken: 933.4738 seconds
Peak Memory Usage: 408.256 MiB


([('Student_1', 'Student_27'),
  ('Student_10', 'Student_8'),
  ('Student_11', 'Student_31'),
  ('Student_12', 'Student_3'),
  ('Student_13', 'Student_34'),
  ('Student_14', 'Student_22'),
  ('Student_15', 'Student_30'),
  ('Student_16', 'Student_37'),
  ('Student_17', 'Student_39'),
  ('Student_18', 'Student_21'),
  ('Student_19', 'Student_38'),
  ('Student_2', 'Student_5'),
  ('Student_20', 'Student_24'),
  ('Student_23', 'Student_29'),
  ('Student_25', 'Student_6'),
  ('Student_26', 'Student_35'),
  ('Student_28', 'Student_4'),
  ('Student_32', 'Student_9'),
  ('Student_33', 'Student_36'),
  ('Student_40', 'Student_7')],
 17.597011035262096,
 24272190,
 933.4737551212311,
 408.255976)

## Focus on speed instead of efficiency (memory usage)

In [ ]:
import pandas as pd
import numpy as np
import time
import tracemalloc
from numba import njit, typed, types

# Define a tuple type for a pair of int64 values.
pair_type = types.Tuple((types.int64, types.int64))

# ============================================================
# Numba-accelerated helper functions
# ============================================================

@njit(cache=True)
def compute_upper_bound(mask, comp, n):
    """
    Given a bitmask 'mask' indicating which students are unmatched,
    compute an upper bound on the additional compatibility score achievable
    from the remaining students.
    """
    additional = 0.0
    for i in range(n):
        if mask & (1 << i):
            best_val = 0.0
            for j in range(n):
                if (mask & (1 << j)) and (i != j):
                    if comp[i, j] > best_val:
                        best_val = comp[i, j]
            additional += best_val
    return additional / 2.0

@njit(cache=True)
def clear_typed_list(typed_list):
    """
    Clears a Numba-typed list by popping all its elements.
    """
    count = len(typed_list)
    for _ in range(count):
        typed_list.pop()

@njit(cache=True)
def search(mask, current_score, comp, n, best_score, best_pairs, current_pairs, memo, allocations):
    """
    Recursive branch and bound search.
    
    Parameters:
      mask: Bitmask (np.int64) representing unmatched students.
      current_score: Accumulated compatibility score.
      comp: n x n NumPy array of compatibility scores.
      n: Total number of students.
      best_score: A 1-element NumPy array (mutable container) for the best score.
      best_pairs: A Numba-typed list (of 2-element tuples of int64) storing the best pairing.
      current_pairs: A Numba-typed list (of 2-element tuples of int64) storing the current pairing.
      memo: A Numba-typed dictionary mapping masks (int64) to float64 upper bounds.
      allocations: A 1-element NumPy array (of int64) used as a counter.
    """
    # Base case: all students paired.
    if mask == 0:
        if current_score > best_score[0]:
            best_score[0] = current_score
            clear_typed_list(best_pairs)
            for idx in range(len(current_pairs)):
                best_pairs.append(current_pairs[idx])
        return

    # Use memoization to prune.
    if mask in memo:
        if current_score + memo[mask] <= best_score[0]:
            return
    else:
        ub_val = compute_upper_bound(mask, comp, n)
        memo[mask] = ub_val
        if current_score + ub_val <= best_score[0]:
            return

    # Choose the first unmatched student (lowest-index bit in mask).
    i = 0
    while i < n:
        if mask & (1 << i):
            break
        i += 1

    new_mask = mask & ~(1 << i)
    
    # Build a NumPy array of candidate partner indices from new_mask.
    candidate_count = 0
    candidate_arr = np.empty(n, dtype=np.int64)
    for j in range(n):
        if new_mask & (1 << j):
            candidate_arr[candidate_count] = j
            candidate_count += 1
    candidate_arr = candidate_arr[:candidate_count]
    
    # Sort candidate indices in descending order of compatibility with student i.
    order_temp = np.argsort(comp[i, candidate_arr])
    m = order_temp.shape[0]
    order = np.empty(m, dtype=np.int64)
    for k in range(m):
        order[k] = order_temp[m - 1 - k]  # Reverse order.
    sorted_candidates = candidate_arr[order]
    
    # Try pairing student i with each candidate.
    for j in sorted_candidates:
        pair_score = comp[i, j]
        next_mask = new_mask & ~(1 << j)
        current_pairs.append((i, j))
        allocations[0] += 1
        search(next_mask, current_score + pair_score, comp, n, best_score, best_pairs, current_pairs, memo, allocations)
        current_pairs.pop()

@njit(cache=True)
def branch_and_bound_numba(comp, n):
    """
    Entry point for the Numba-accelerated branch and bound search.
    
    Returns a tuple:
      (best total compatibility score, best pairing as a typed list, allocation count)
    """
    best_score = np.array([-1e9], dtype=np.float64)  # 1-element array for best score.
    best_pairs = typed.List.empty_list(pair_type)
    current_pairs = typed.List.empty_list(pair_type)
    memo = typed.Dict.empty(key_type=types.int64, value_type=types.float64)
    allocations = np.zeros(1, dtype=np.int64)  # 1-element counter.
    full_mask = np.int64((1 << n) - 1)  # Cast to np.int64.
    search(full_mask, 0.0, comp, n, best_score, best_pairs, current_pairs, memo, allocations)
    return best_score[0], best_pairs, allocations[0]

# ============================================================
# Python wrapper function
# ============================================================
def solve_roommate_matching_improved(csv_path, method='branch_and_bound'):
    """
    Reads compatibility data from CSV and computes the optimal pairing using
    a branch and bound algorithm with a bitmask-based representation and Numba acceleration.
    
    Returns:
      - best_pairing_named: List of tuples (student names) for the optimal pairing.
      - best_score: Total compatibility score.
      - allocations: Number of recursive branch allocations (nodes expanded).
      - run_time: Total execution time in seconds.
      - peak_memory: Peak memory usage in MiB.
    """
    # Load CSV data.
    df = pd.read_csv(csv_path)
    
    # Build a sorted list of unique student names.
    student_names = sorted(list(set(df['Student 1'].tolist() + df['Student 2'].tolist())))
    n = len(student_names)
    student_to_index = {name: i for i, name in enumerate(student_names)}
    index_to_student = {i: name for i, name in enumerate(student_names)}
    
    # Build an n x n compatibility matrix as a NumPy array.
    comp = np.zeros((n, n), dtype=np.float64)
    for _, row in df.iterrows():
        i = student_to_index[row['Student 1']]
        j = student_to_index[row['Student 2']]
        score = row['Compatibility Score']
        comp[i, j] = score
        comp[j, i] = score  # Ensure symmetry.
    
    # Start timing and memory tracking.
    tracemalloc.start()
    start_time = time.time()
    
    best, best_pairs, allocs = branch_and_bound_numba(comp, n)
    
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    run_time = end_time - start_time
    peak_memory = peak / 10**6  # Convert bytes to MiB.
    
    # Convert best_pairs (typed list of index pairs) to pairs of student names.
    best_pairing_named = [(index_to_student[p[0]], index_to_student[p[1]]) for p in best_pairs]
    
    print("=== Improved Branch and Bound (Numba) ===")
    print(f"Optimal Pairing: {best_pairing_named}")
    print(f"Total Compatibility Score: {best}")
    print(f"Number of Allocations (nodes expanded): {allocs}")
    print(f"Time Taken: {run_time:.4f} seconds")
    print(f"Peak Memory Usage: {peak_memory:.3f} MiB")
    
    return best_pairing_named, best, allocs, run_time, peak_memory

In [129]:
solve_roommate_matching_improved("data/compatibility_40.csv")

=== Improved Branch and Bound (Numba) ===
Optimal Pairing: [('Student_1', 'Student_27'), ('Student_10', 'Student_8'), ('Student_11', 'Student_31'), ('Student_12', 'Student_3'), ('Student_13', 'Student_34'), ('Student_14', 'Student_22'), ('Student_15', 'Student_30'), ('Student_16', 'Student_37'), ('Student_17', 'Student_39'), ('Student_18', 'Student_21'), ('Student_19', 'Student_38'), ('Student_2', 'Student_5'), ('Student_20', 'Student_24'), ('Student_23', 'Student_29'), ('Student_25', 'Student_6'), ('Student_26', 'Student_35'), ('Student_28', 'Student_4'), ('Student_32', 'Student_9'), ('Student_33', 'Student_36'), ('Student_40', 'Student_7')]
Total Compatibility Score: 17.597011035262096
Number of Allocations (nodes expanded): 24272079
Time Taken: 11.7853 seconds
Peak Memory Usage: 0.008 MiB


([('Student_1', 'Student_27'),
  ('Student_10', 'Student_8'),
  ('Student_11', 'Student_31'),
  ('Student_12', 'Student_3'),
  ('Student_13', 'Student_34'),
  ('Student_14', 'Student_22'),
  ('Student_15', 'Student_30'),
  ('Student_16', 'Student_37'),
  ('Student_17', 'Student_39'),
  ('Student_18', 'Student_21'),
  ('Student_19', 'Student_38'),
  ('Student_2', 'Student_5'),
  ('Student_20', 'Student_24'),
  ('Student_23', 'Student_29'),
  ('Student_25', 'Student_6'),
  ('Student_26', 'Student_35'),
  ('Student_28', 'Student_4'),
  ('Student_32', 'Student_9'),
  ('Student_33', 'Student_36'),
  ('Student_40', 'Student_7')],
 17.597011035262096,
 24272079,
 11.785269021987915,
 0.008104)

In [130]:
solve_roommate_matching_improved("data/compatibility_50.csv")

=== Improved Branch and Bound (Numba) ===
Optimal Pairing: [('Student_1', 'Student_35'), ('Student_10', 'Student_46'), ('Student_11', 'Student_4'), ('Student_12', 'Student_48'), ('Student_13', 'Student_17'), ('Student_14', 'Student_3'), ('Student_15', 'Student_7'), ('Student_16', 'Student_37'), ('Student_18', 'Student_34'), ('Student_19', 'Student_5'), ('Student_2', 'Student_22'), ('Student_20', 'Student_23'), ('Student_21', 'Student_24'), ('Student_25', 'Student_39'), ('Student_26', 'Student_49'), ('Student_27', 'Student_8'), ('Student_28', 'Student_44'), ('Student_29', 'Student_6'), ('Student_30', 'Student_9'), ('Student_31', 'Student_45'), ('Student_32', 'Student_33'), ('Student_36', 'Student_43'), ('Student_38', 'Student_47'), ('Student_40', 'Student_42'), ('Student_41', 'Student_50')]
Total Compatibility Score: 22.377550358731888
Number of Allocations (nodes expanded): 7692404
Time Taken: 8.3650 seconds
Peak Memory Usage: 0.011 MiB


([('Student_1', 'Student_35'),
  ('Student_10', 'Student_46'),
  ('Student_11', 'Student_4'),
  ('Student_12', 'Student_48'),
  ('Student_13', 'Student_17'),
  ('Student_14', 'Student_3'),
  ('Student_15', 'Student_7'),
  ('Student_16', 'Student_37'),
  ('Student_18', 'Student_34'),
  ('Student_19', 'Student_5'),
  ('Student_2', 'Student_22'),
  ('Student_20', 'Student_23'),
  ('Student_21', 'Student_24'),
  ('Student_25', 'Student_39'),
  ('Student_26', 'Student_49'),
  ('Student_27', 'Student_8'),
  ('Student_28', 'Student_44'),
  ('Student_29', 'Student_6'),
  ('Student_30', 'Student_9'),
  ('Student_31', 'Student_45'),
  ('Student_32', 'Student_33'),
  ('Student_36', 'Student_43'),
  ('Student_38', 'Student_47'),
  ('Student_40', 'Student_42'),
  ('Student_41', 'Student_50')],
 22.377550358731888,
 7692404,
 8.36504602432251,
 0.010544)

In [132]:
solve_roommate_matching_improved("data/compatibility_50.csv")

=== Improved Branch and Bound (Numba) ===
Optimal Pairing: [('Student_1', 'Student_35'), ('Student_10', 'Student_46'), ('Student_11', 'Student_4'), ('Student_12', 'Student_48'), ('Student_13', 'Student_17'), ('Student_14', 'Student_3'), ('Student_15', 'Student_7'), ('Student_16', 'Student_37'), ('Student_18', 'Student_34'), ('Student_19', 'Student_5'), ('Student_2', 'Student_22'), ('Student_20', 'Student_23'), ('Student_21', 'Student_24'), ('Student_25', 'Student_39'), ('Student_26', 'Student_49'), ('Student_27', 'Student_8'), ('Student_28', 'Student_44'), ('Student_29', 'Student_6'), ('Student_30', 'Student_9'), ('Student_31', 'Student_45'), ('Student_32', 'Student_33'), ('Student_36', 'Student_43'), ('Student_38', 'Student_47'), ('Student_40', 'Student_42'), ('Student_41', 'Student_50')]
Total Compatibility Score: 22.377550358731888
Number of Allocations (nodes expanded): 7692404
Time Taken: 7.3733 seconds
Peak Memory Usage: 0.011 MiB


([('Student_1', 'Student_35'),
  ('Student_10', 'Student_46'),
  ('Student_11', 'Student_4'),
  ('Student_12', 'Student_48'),
  ('Student_13', 'Student_17'),
  ('Student_14', 'Student_3'),
  ('Student_15', 'Student_7'),
  ('Student_16', 'Student_37'),
  ('Student_18', 'Student_34'),
  ('Student_19', 'Student_5'),
  ('Student_2', 'Student_22'),
  ('Student_20', 'Student_23'),
  ('Student_21', 'Student_24'),
  ('Student_25', 'Student_39'),
  ('Student_26', 'Student_49'),
  ('Student_27', 'Student_8'),
  ('Student_28', 'Student_44'),
  ('Student_29', 'Student_6'),
  ('Student_30', 'Student_9'),
  ('Student_31', 'Student_45'),
  ('Student_32', 'Student_33'),
  ('Student_36', 'Student_43'),
  ('Student_38', 'Student_47'),
  ('Student_40', 'Student_42'),
  ('Student_41', 'Student_50')],
 22.377550358731888,
 7692404,
 7.373286962509155,
 0.010544)